In [ ]:
import torch
import sys
import os
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.cm as cm
import scipy
from datetime import datetime
import json

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)
print(sys.path)

import Learning.autoencoders as autoencoders
import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.transforms as transforms
import Double_Pendulum.dynamics as dynamics
import Plotting.pendulum_plot as pendulum_plot

import Double_Pendulum.normal_form as normal_form

from functools import partial



%load_ext autoreload
%autoreload 2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
rp = robot_parameters.LUMPED_PARAMETERS
plotter = pendulum_plot.Anim_plotter(rp)
neural_net = False
rp["m1"] = 0.
fix_q = True
model_cw = False
model_shifting = True

if fix_q and not neural_net:
    fix_q = False
    print("Analytic transform used, so fix_q set to False. Did you check your inputs properly?")

if neural_net:
    model = autoencoders.Autoencoder_double(rp).to(device)
    model_location = '../Learning/Models/Lumped_Mass_ccw_12k_epochs.pth'
    model.load_state_dict(torch.load(model_location, weights_only=True))
else:
    model_location = "N/A"
    model = autoencoders.Analytic_transformer(rp)

In [ ]:
xy_start = torch.tensor([4., -1.9]).requires_grad_().to(device)
q_d_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)
q_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)


xy_des_real = torch.tensor([2.1, -2.]).requires_grad_().to(device)
q_d_des = torch.tensor([[0., 0.]]).requires_grad_().to(device)

In [ ]:
k_spring = 0.05

In [ ]:
q_start = transforms.inverse_kinematics(xy_start, rp, is_clockwise=model_cw).unsqueeze(0)
is_clockwise_start = transforms.check_clockwise(q_start.squeeze(0))

q_des = transforms.inverse_kinematics(xy_des_real, rp, is_clockwise=model_cw).unsqueeze(0)
th_start = model.encoder_vmap(q_start)
th_d_start = (model.jacobian_enc(q_start) @ q_d_start.T).T
th_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device) #TODO: Obtain accelerations in thetas. Do I care?

is_clockwise_des = transforms.check_clockwise(q_des.squeeze(0))

th_des = model.encoder_vmap(q_des)
th_d_des = (model.jacobian_enc(q_des) @ q_d_des.T).T


q_des_hat = model.decoder_vmap(th_des, is_clockwise_des)
q_d_des_hat = (model.jacobian_dec(th_des, clockwise=is_clockwise_des) @ th_d_des.T).T
xy_des_est, _ = transforms.forward_kinematics(rp, q_des_hat[0])

print(q_start)

In [ ]:
J_h_inv_des = model.jacobian_dec(th_des, is_clockwise_des).squeeze(0)
J_h_inv_trans_des = torch.transpose(J_h_inv_des, 0, 1)

M_q_des, C_q_des, G_q_des = dynamics.dynamical_matrices(rp, q_des_hat.squeeze(0), q_d_des_hat.squeeze(0))
G_q_des = dynamics.add_spring_force_G_q(rp, q_des_hat, G_q_des, k_spring)
M_th_des, C_th_des, G_th_des = transforms.transform_dynamical_from_inverse(M_q_des, C_q_des, G_q_des, th_des, th_d_des, J_h_inv_des, J_h_inv_trans_des)

K = torch.tensor([[10., 100., 100., 100.]]).to(device)
Y_des_u = normal_form.calculate_Y(th_des, th_d_des, M_th_des, C_th_des, G_th_des, device)
Y_des = torch.tensor([[Y_des_u[0,0]], [0], [0], [0]])
print("Y_des_u:\n", Y_des_u)
print("Y_des:\n", Y_des)

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

th_series, th_d_series, th_dd_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_est_series, q_d_est_series, q_dd_est_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_real_series, q_d_real_series, q_dd_real_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
Y_series = torch.empty((0,4)).to(device)
u_series = torch.empty((0,1)).to(device)


dt = 0.01
t_end = 5
t_series = torch.arange(0, t_end, dt)

th = th_start
th_d = th_d_start
th_dd = th_dd_start

q_est = model.decoder(th_start, clockwise=is_clockwise_start)
q_d_est = (model.jacobian_dec(th_start, clockwise=is_clockwise_start) @ th_d_start.T).T
q_dd_est = torch.zeros((1,2))

q_real, q_d_real, q_dd_real = q_start, q_d_start, q_dd_start

is_clockwise = transforms.check_clockwise(q_start.squeeze(0))


if neural_net and fix_q:
    note_string = "NOTE: The controller expects the NN to be trained on " + ("COUNTER" if not model_cw else "") + "CLOCKWISE data."
    print(note_string)
if model_shifting:
    print("NOTE: The controller expects points in q to be shifted (to make q-space unimodal).")

for t in torch.arange(0, t_end, dt):
    t_string = "Time: [" + str(t.item().__round__(3)) + "/" + str(t_end) + ".0]"
    print(t_string)


    is_clockwise = transforms.check_clockwise(q_real.squeeze(0))
    if fix_q and model_cw != is_clockwise:
        print("fixing value to", "clockwise" if model_cw else "counterclockwise")
        q = transforms.flip_q(rp, q_real.squeeze(0), model_cw).unsqueeze(0)
        q_d = transforms.flip_q_d(rp, q_real.squeeze(0), q_d_real, model_cw)
    else:
        q = q_real
        q_d = q_d_real

    if model_shifting:
        q = transforms.shift_q(q, clockwise=model_cw)

    th = model.encoder_vmap(q)
    th_d = (model.jacobian_enc(q) @ q_d.T).T

    q_hat = model.decoder_vmap(th, clockwise=model_cw)
    q_d_hat = (model.jacobian_dec(th) @ th_d.T).T
    
    """ Obtain Jacobian, dynamical matrices"""

    is_clockwise = transforms.check_clockwise(q_hat.squeeze(0))
    
    J_h_inv = model.jacobian_dec(th, is_clockwise).squeeze(0)
    J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

    print("q:", q)
    print("q_d:", q_d)
    print("q_hat:", q_hat)
    print("q_d_hat:", q_d_hat)
    print("th:", th)
    print("th_d:", th_d)


    M_q_est, C_q_est, G_q_est = dynamics.dynamical_matrices(rp, q_hat.squeeze(0), q_d_hat.squeeze(0))
    G_q_est = dynamics.add_spring_force_G_q(rp, q_hat, G_q_est, k_spring)
    A_q_est = dynamics.input_matrix(rp, q_hat.squeeze(0))


    """ Feed-forward simulation of the system, not on real dynamics """
    M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q_est, C_q_est, G_q_est, th, th_d, J_h_inv, J_h_inv_trans)
    A_th = transforms.transform_input_matrix_from_inverse_trans(A_q_est, J_h_inv_trans, device)

    Y = normal_form.calculate_Y(th, th_d, M_th, C_th, G_th, device)
    alpha, beta = normal_form.calculate_alpha_beta(th, th_d, M_th, C_th, G_th, A_th, Y)

    print("alpha:", alpha)
    print("beta:", beta)
    v = normal_form.calculate_v(Y, Y_des, K)
    u = normal_form.calculate_u(alpha, beta, v)
    print("Y:", Y)
    print("v:", v)
    print("u:", u)

    """ Update the real system and apply latent control input. """

    M_q_real, C_q_real, G_q_real = dynamics.dynamical_matrices(rp, q_real.squeeze(0), q_d_real.squeeze(0))
    G_q_real = dynamics.add_spring_force_G_q(rp, q_real, G_q_real, k_spring)
    A_q_real = dynamics.input_matrix(rp, q_real.squeeze(0))

    tau_q_real = A_q_real * u
    q_dd_real = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d_real).T)- G_q_real)).T  #  - C_damp @ ((q_d_real).T) 
    q_d_real = q_d_real + q_dd_real * dt
    q_real = q_real + q_d_real * dt
    q_real = transforms.wrap_to_pi(q_real)
    

    q_real_shifted = transforms.shift_q(q_real, clockwise=model_cw)
    th = model.encoder_vmap(q_real_shifted)
    q_est = model.decoder_vmap(th, clockwise=transforms.check_clockwise(q_real.squeeze(0)))
    q_est = transforms.wrap_to_pi(q_est)
    th_d = (model.jacobian_enc(q_real) @ q_d_real.T).T
    q_d_est = (model.jacobian_dec(th, clockwise=is_clockwise) @ th_d.T).T
    q_dd_est = torch.tensor([[0., 0.]]).requires_grad_().to(device) #TODO: Implement with Jacobian derivative


    """ Store data for plotting """

    th_series = torch.cat((th_series, th.detach()), dim=0)
    th_d_series = torch.cat((th_d_series, th_d.detach()), dim=0)
    th_dd_series = torch.cat((th_dd_series, th_dd.detach()), dim=0)
    
    q_real_series = torch.cat((q_real_series, q_real.detach()), dim=0)
    q_d_real_series = torch.cat((q_d_real_series, q_d_real.detach()), dim=0)
    q_dd_real_series = torch.cat((q_dd_real_series, q_dd_real.detach()), dim=0)

    q_est_series = torch.cat((q_est_series, q_est.detach()), dim=0)
    q_d_est_series = torch.cat((q_d_est_series, q_d_est.detach()), dim=0)
    q_dd_est_series = torch.cat((q_dd_est_series, q_dd_est.detach()), dim=0) 

    u_series = torch.cat((u_series, u.detach()), dim=0)
    Y_series = torch.cat((Y_series, Y.detach().T))
    print("")

    if torch.isnan(q[0,0]):
        break

    
    


In [ ]:
xy_real_series = torch.empty((0,2)).to(device)
xy_est_series = torch.empty((0,2)).to(device)
xy_naive_series = torch.empty((0,2)).to(device)

#for q_real, q_est, q_naive in zip(q_real_series, q_est_series, q_naive_series):
for q_real, q_est in zip(q_real_series, q_est_series):
    xy_real, _ = transforms.forward_kinematics(rp, q_real)
    xy_real_series = torch.cat((xy_real_series, xy_real.unsqueeze(0)))
    xy_est, _ = transforms.forward_kinematics(rp, q_est)
    xy_est_series = torch.cat((xy_est_series, xy_est.unsqueeze(0)))
    #xy_naive, _ = transforms.forward_kinematics(rp, q_naive)
    #xy_naive_series = torch.cat((xy_naive_series, xy_naive.unsqueeze(0)))


In [ ]:
save_dir = f"non-collocated_results/simulation_{timestamp}"
os.makedirs(save_dir, exist_ok=True)
print("save_dir:", save_dir)

stride = 1
pos_end_q_real, pos_elbow_q_real = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_real_series[::stride])
pos_end_q_est, pos_elbow_q_est = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_est_series[::stride])

frames_q_real = plotter.frame_pendulum(pos_end_q_real, pos_elbow_q_real)
frames_q_est = plotter.frame_pendulum(pos_end_q_est, pos_elbow_q_est)

data_real = {
    "frames": frames_q_real,
    "times": dt,
    "name": "q_real", 
    "arm_color": "tab:blue",
    "act_color": "tab:cyan"
}

data_est = {
    "frames": frames_q_est,
    "times": dt,
    "name": "q_est",
    "arm_color": "tab:orange", 
    "act_color": "tab:red"
} 

ref_pos_real = {
    "pos": xy_des_real,
    "name": "real",
    "color": "tab:blue"
}

ref_pos_est = {
    "pos": xy_des_est,
    "name": "est",
    "color": "tab:orange"
}



ref_poss = [ref_pos_real]#, ref_pos_est]

frames_data = [data_real, data_est]

name_rp = "RP:(" + str(rp["xa"]) + "," + str(rp["ya"]) + ")_"
name_ref = "ref:(" + str(xy_des_real[0].item()) + "," + str(xy_des_real[1].item()) + ")_"

if neural_net:
    model_type = "NN"
else:
    model_type = "AL"

file_name = "t_end:[" + str(t_end) + "]_dt:[" + str(dt) + "]_stride:[" + str(stride) + "]_0.mp4"
file_counter = 0

output_path = os.path.join(save_dir, file_name)

while os.path.isfile(output_path):
    print("file name already exists")
    file_counter += 1
    file_name = file_name[:-6] + "_" + str(file_counter) + ".mp4"
    output_path = os.path.join(save_dir, file_name)
    
plotter.animate_pendulum(frames_data, ref_poss=ref_poss, plot_actuator=True, save_path=output_path, fps = 0.1/(dt*stride), dt = dt*stride)
#plotter.animate_pendulum(frames_data, ref_pos=None, plot_actuator=False, file_name=file_name, fps = 1/(dt*stride), dt = dt*stride)
